# Let's get some free city data for Albuquerque, NM and make a heatmap of where in town most Breaking Bad filming took place

In [ ]:
import geopandas as gp
import pathlib as pl
import fiona
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default


In [ ]:
# data from https://www.cabq.gov/abq-data/

In [ ]:
datapath = pl.Path('./data/geopandas/abq/')

In [ ]:
gdf = gp.read_file(datapath / 'abq_films.geojson')
gdf = gdf.loc[gdf.geometry != None] # there are some missing geometries, so we can skip those
gdf['Title'] = gdf['Title'].apply(lambda x:x.strip()) # let's strip off extra spaces from the Titles field
gdf.Title = ['In Plain Sight' if "Plain Sight" in i else i for i in gdf.Title]
gdf['Year'] = [i.year for i in gdf.ShootDate]
gdf = gdf.drop(columns='ShootDate') # something with this 

### what are the top 20 projects filming in ABQ?

In [ ]:
gdf.groupby('Title').count()['Type'].sort_values(ascending=False)[:20].plot.bar()

In [ ]:
gdf.loc[gdf.Title=='Breaking Bad'].explore()

In [ ]:
gdf['Title'].unique()

In [ ]:
gdf['Type'].unique()

## read in the zone atlas

In [ ]:
gdf_pg = gp.read_file(datapath / 'zoneatlaspagegrid.kmz')
gdf_pg.explore()

In [ ]:
title_to_plot = 'Breaking Bad'

In [ ]:
gdf_pg = gp.read_file(datapath / 'zoneatlaspagegrid.kmz')
gdf_pg = gdf_pg.merge(gdf_pg.sjoin(
    gdf.loc[gdf.Title==title_to_plot].to_crs(gdf_pg.crs)).groupby(
    'Name').count().geometry.rename('points').reset_index())

In [ ]:
m = gdf_pg[['points','geometry']].explore(column='points', cmap='magma_r')
gdf.loc[gdf.Title==title_to_plot][['Site', 'geometry']].explore(m=m)
